# Examen del 2do Parcial

1. Utilizando el dataset de _Coactivation_matrix.mat_:
- Calcule el coeficiente de mundo pequeño
- Calcule las comunidades del grafo
- Calcule los hub
- Calcule la eficiencia global
- Calcule la eficiencia local
- Determine el grado de cada nodo

2. Utilizando el dataset de _Coactivation_matrix.mat_:
- Generar el mapa de calor de cada matriz de conectividad
- Generar la distribución de datos de cada matriz de conectividad
- ¿Qué valor se encuentra en el percentil 0.25, 0.5 y 0.75 de la matriz de conectividad?
- Cree los 3 grafos 2D filtrando la matriz de conectividad con los valores dados por los percentiles del ejercicio anterior

3. Utilizando el dataset de _Coactivation_matrix.mat_:
- Genere el grafo 3D
- Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado
- Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot

4. Utilizando el dataset de _chb01_01.edf_:
- Calcule el coeficiente de mundo pequeño
- Calcule las comunidades del grafo
- Calcule los hub
- Calcule la eficiencia global
- Calcule la eficiencia local
- Determine el grado de cada nodo

5. Utilizando el dataset de _chb01_01.edf_:
- Genere el grafo 3D
- Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado
- Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot


In [16]:
import numpy as np
import pandas as pd
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
import networkx.algorithms.smallworld as sw
import networkx.algorithms.community as nx_comm
import mne
from scipy.signal import coherence, hilbert

In [18]:
datos_ruta = r"C:\Users\maria\OneDrive\Documentos\MATLAB\Coactivation_matrix.mat"
datos = scipy.io.loadmat(datos_ruta)
data = datos["Coactivation_matrix"]
df = pd.DataFrame(data)
df = df.fillna(0)
G = nx.from_numpy_array(data)

In [20]:
C = nx.average_clustering(G)
L = nx.average_shortest_path_length(G)
C, L
print("Coef cluster:", C)
print("Ruta más corta:", L)
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
G_random = nx.gnm_random_graph(n_nodes, n_edges)
clustering_coefficient_random = nx.average_clustering(G_random)
print("Coef cluster (random):", clustering_coefficient_random)
if nx.is_connected(G_random):
    avg_shortest_path_length_random = nx.average_shortest_path_length(G_random)
    print("Ruta más corta (random):", avg_shortest_path_length_random)
if avg_shortest_path_length_random > 0 and L > 0:
    small_world_coefficient = (C / clustering_coefficient_random) / (L / avg_shortest_path_length_random)
    print("Coef mundo pequeño:", small_world_coefficient)

Coef cluster: 0.3844533292242755
Ruta más corta: 2.2148737961545844
Coef cluster (random): 0.09179016074376502
Ruta más corta (random): 1.9123831833191438
Coef mundo pequeño: 3.6163747362502545


In [ ]:
from networkx.algorithms import community
comunidades = list(community.greedy_modularity_communities(G))
print(f"Ncomunidades: {len(comunidades)}")
for i, c in enumerate(comunidades):
    print(f"Comunidad {i+1} ({len(c)} nodos): {sorted(list(c))[:10]}")

In [ ]:
centralidad = nx.degree_centrality(G)
print("hub centralidad")
C=sorted(centralidad.items(), key=lambda item: item[1], reverse=True)
print(C[:10])
intermediacion=nx.betweenness_centrality(G, weight='weight')
print("\n hub intermediación")
In=sorted(intermediacion.items(), key=lambda item: item[1], reverse=True)
print(In[:10])

In [ ]:
degrees = dict(G.degree())
avg_degree = np.mean(list(degrees.values()))
hubs = [node for node, degree in degrees.items() if degree > avg_degree]
print("Hubs degree - average):", hubs)

In [ ]:
def global_efficiency(G):
    n = len(G)
    if n <= 1:
        return 0
    lengths = dict(nx.all_pairs_shortest_path_length(G))
    s = 0
    for u in lengths:
        for v, d in lengths[u].items():
            if u != v:
                s += 1 / d
    return s / (n * (n - 1))
def local_efficiency(G):
    effs = []
    for v in G:
        vecinos = list(G.neighbors(v))
        if len(vecinos) >= 2:
            sub = G.subgraph(vecinos)
            effs.append(global_efficiency(sub))
    return np.mean(effs)
E_global = global_efficiency(G)
E_local = local_efficiency(G)
print(f"Ef global: {E_global:.3f}")
print(f"Ef local:  {E_local:.3f}")

In [ ]:
degree_dict = dict(G.degree())
degree_dict

In [ ]:
sfreq_target = 256       
fmin, fmax = 8, 13 
corr_matrix = np.corrcoef(data)
corr_df = pd.DataFrame(corr_matrix)
n_channels=638
coh_matrix = np.zeros((n_channels, n_channels))
for i in range(n_channels):
    for j in range(n_channels):
        f, Cxy = coherence(data[i], data[j], fs=sfreq_target, nperseg=sfreq_target*2)
        mask = (f >= fmin) & (f <= fmax)
        coh_matrix[i, j] = np.mean(Cxy[mask])
coh_df = pd.DataFrame(coh_matrix)
analytic_signal = hilbert(data)
phase_data = np.angle(analytic_signal)
plv_matrix = np.zeros((n_channels, n_channels))
for i in range(n_channels):
    for j in range(n_channels):
        phase_diff = phase_data[i] - phase_data[j]
        plv_matrix[i, j] = np.abs(np.sum(np.exp(1j * phase_diff)) / phase_diff.size)
plv_df = pd.DataFrame(plv_matrix)

In [ ]:
plt.figure(figsize=(8, 2))
co= sns.heatmap(corr_df.values,
                 annot=False, cmap='GnBu', fmt=".2f")
plt.show()
plt.figure(figsize=(8, 6))
val_corr = corr_df.values[np.triu_indices_from(corr_df, k=1)]
plt.hist(val_corr, bins=45)
p25 = np.percentile(val_corr, 25)  
p50 = np.percentile(val_corr, 50)  
p75 = np.percentile(val_corr, 75)  
print(f"percentil 25: {p25:.4f}")
print(f"percentil 50: {p50:.4f}")
print(f"percentil 75: {p75:.4f}")
percentiles = {'25': p25, '50': p50, '75': p75}
filtrado_p = {}
for name, threshold in percentiles.items():
    G_filt_p = nx.Graph()
    num_nodes = corr_df.shape[0]
    G_filt_p.add_nodes_from(range(num_nodes))
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            weight = corr_df.iloc[i, j]
            if abs(weight) > threshold: 
                G_filt_p.add_edge(i, j, weight=weight)
    graph_key = f"pearson_{name.replace(' ', '_').lower()}"
    filtrado_p[graph_key] = G_filt_p
    print(f"Created graph '{graph_key}' with {G_filt_p.number_of_nodes()} nodes and {G_filt_p.number_of_edges()} edges.")
for graph_key, G_to_draw in filtrado_p.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw) 
    nx.draw(G_to_draw, pos, with_labels=False, node_size=20, edge_color='gray', alpha=0.6)
    plt.title(f'Filt Graph: {graph_key.replace("_", " ").title()}')
    plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
coh = sns.heatmap(coh_df.values,
                 annot=False, cmap='GnBu', fmt=".2f")
plt.show()
plt.figure(figsize=(8, 6))
val_coh = corr_df.values[np.triu_indices_from(coh_df, k=1)]
plt.hist(val_coh, bins=45)
p25_c = np.percentile(val_coh, 25)  
p50_c = np.percentile(val_coh, 50)  
p75_c = np.percentile(val_coh, 75)  
print(f"Percentil 25: {p25_c:.4f}")
print(f"Percentil 50: {p50_c:.4f}")
print(f"Percentil 75: {p75_c:.4f}")
percentiles_c = {'25': p25_c, '50': p50_c, '75': p75_c}
filtrado_c = {}
for name, threshold in percentiles_c.items():
    G_filt_c = nx.Graph()
    num_nodes_c = coh_df.shape[0]
    G_filt_c.add_nodes_from(range(num_nodes_c))
    for i in range(num_nodes_c):
        for j in range(i + 1, num_nodes_c):
            weight_c = coh_df.iloc[i, j]
            if abs(weight_c) > threshold: 
                G_filt_c.add_edge(i, j, weight_c=weight_c)
    graph_key_c = f"coherencia espectral_{name.replace(' ', '_').lower()}"
    filtrado_c[graph_key_c] = G_filt_c
    print(f"Created graph '{graph_key_c}' with {G_filt_c.number_of_nodes()} nodes and {G_filt_c.number_of_edges()} edges.")
for graph_key, G_to_draw in filtrado_c.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw) 
    nx.draw(G_to_draw, pos, with_labels=False, node_size=20, edge_color='gray', alpha=0.6)
    plt.title(f'Filtered Graph: {graph_key.replace("_", " ").title()}')
    plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
plv = sns.heatmap(plv_df.values,
                 annot=False, cmap='GnBu', fmt=".2f")
plt.show()
plt.figure(figsize=(8, 6))
val_plv = corr_df.values[np.triu_indices_from(plv_df, k=1)]
plt.hist(val_plv, bins=45)
p25_p = np.percentile(val_plv, 25)  
p50_p = np.percentile(val_plv, 50)  
p75_p = np.percentile(val_plv, 75)  
print(f"Percentil 25: {p25_p:.4f}")
print(f"Percentil 50: {p50_p:.4f}")
print(f"Percentil 75: {p75_p:.4f}")
percentiles_p = {'25': p25_p, '50': p50_p, '75': p75_p}
filtrado_p = {}
for name, threshold in percentiles_p.items():
    G_filt_p = nx.Graph()
    num_nodes_p = coh_df.shape[0]
    G_filt_p.add_nodes_from(range(num_nodes_p))

    for i in range(num_nodes_p):
        for j in range(i + 1, num_nodes_p):
            weight_p = plv_df.iloc[i, j]
            if abs(weight_p) > threshold: 
                G_filt_c.add_edge(i, j, weight_p=weight_p)
    graph_key_p = f"phase locking value_{name.replace(' ', '_').lower()}"
    filtrado_p[graph_key_p] = G_filt_p
    print(f"Created graph '{graph_key_p}' with {G_filt_p.number_of_nodes()} nodes and {G_filt_p.number_of_edges()} edges.")
for graph_key, G_to_draw in filtrado_p.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw) 
    nx.draw(G_to_draw, pos, with_labels=False, node_size=20, edge_color='blue', alpha=0.6)
    plt.title(f'Filtered Graph: {graph_key.replace("_", " ").title()}')
    plt.show()

In [ ]:
import matplotlib.cm as cm
coords = datos['Coord'] 
G_3d = nx.from_numpy_array(data)
degrees_3d = dict(G_3d.degree())
node_size_factor = 5 
node_sizes = [degrees_3d[node] * node_size_factor for node in G_3d.nodes()]
edge_weights = np.array([G_3d[u][v]['weight'] for u, v in G_3d.edges()])
if np.max(edge_weights) != np.min(edge_weights):
    norm_edge_weights = (edge_weights - np.min(edge_weights)) / (np.max(edge_weights) - np.min(edge_weights))
else:
    norm_edge_weights = np.zeros_like(edge_weights) 
cmap = cm.get_cmap('hot')
edge_colors = cmap(norm_edge_weights)
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
node_xyz = np.array([coords[i] for i in G_3d.nodes()])
ax.scatter(node_xyz[:, 0], node_xyz[:, 1], node_xyz[:, 2], s=node_sizes, c='skyblue', alpha=0.8)
for i, (u, v) in enumerate(G_3d.edges()):
    x = np.array([coords[u, 0], coords[v, 0]])
    y = np.array([coords[u, 1], coords[v, 1]])
    z = np.array([coords[u, 2], coords[v, 2]])
    ax.plot(x, y, z, color=edge_colors[i], alpha=0.5)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('3DG Coactivation Matrix')
plt.show()

In [ ]:
import mne
raw = mne.io.read_raw_edf(r"C:\Users\maria\OneDrive\Documentos\MATLAB\chb01_01.edf", preload=True)
print(raw)
print(raw.info)

In [ ]:
import bct
eeg_channels = [ch for ch in raw.ch_names if 'EEG' in ch];
raw_eeg = raw.copy().pick_channels(eeg_channels, ordered=True);
variances = raw_eeg.get_data().var(axis=1);
variance_threshold = 1e-10
low_variance_channels = [raw_eeg.ch_names[i] for i, var in enumerate(variances) if var < variance_threshold];
raw_eeg.drop_channels(low_variance_channels);
raw_eeg.set_eeg_reference('average', projection=True);
raw_eeg.apply_proj();
l_freq, h_freq = 1, 40
raw_eeg.filter(l_freq, h_freq, fir_design='firwin');
preprocessed_data = raw_eeg.get_data();
connectivity_matrix = np.corrcoef(preprocessed_data);
connectivity_matrix_df = pd.DataFrame(connectivity_matrix);
connectivity_matrix_df.fillna(0, inplace=True);
connectivity_matrix_df.replace([np.inf, -np.inf], 0, inplace=True);
G_edf = nx.from_numpy_array(connectivity_matrix_df.values);
edf_degrees = dict(G_edf.degree())
print("Grados de nodos:", edf_degrees)
edf_global_efficiency = bct.efficiency_wei(connectivity_matrix_df.values)
print("Ef global:", edf_global_efficiency)
edf_local_efficiency = bct.efficiency_wei(connectivity_matrix_df.values, local=True)
print("Ef local:", edf_local_efficiency)
edf_clustering_coefficient = nx.average_clustering(G_edf)
print("Coef cluster:", edf_clustering_coefficient)
if nx.is_connected(G_edf):
    edf_avg_shortest_path_length = nx.average_shortest_path_length(G_edf)
    print("ruta más corta:", edf_avg_shortest_path_length)

In [ ]:
n_nodes_edf = G_edf.number_of_nodes()
n_edges_edf = G_edf.number_of_edges()
G_random_edf = nx.gnm_random_graph(n_nodes_edf, n_edges_edf)
clustering_coefficient_random_edf = nx.average_clustering(G_random_edf)
print("Coef cluster:", clustering_coefficient_random_edf)
if nx.is_connected(G_random_edf):
    avg_shortest_path_length_random_edf = nx.average_shortest_path_length(G_random_edf)
    print("Ruta más corta:", avg_shortest_path_length_random_edf)
if avg_shortest_path_length_random_edf > 0 and edf_avg_shortest_path_length > 0 and clustering_coefficient_random_edf > 0:
    small_world_coefficient_edf = (edf_clustering_coefficient / clustering_coefficient_random_edf) / (edf_avg_shortest_path_length / avg_shortest_path_length_random_edf)
    print("Coef mundo pequeño:", small_world_coefficient_edf)

import community.community_louvain as community_louvain
G_edf_abs = nx.from_numpy_array(np.abs(connectivity_matrix_df.values))
edf_partition = community_louvain.best_partition(G_edf_abs)
print("comunidades:", edf_partition)
edf_degrees_abs = dict(G_edf_abs.degree())
edf_avg_degree_abs = np.mean(list(edf_degrees_abs.values()))
edf_hubs_abs = [node for node, degree in edf_degrees_abs.items() if degree > edf_avg_degree_abs]
print("Hubs", edf_hubs_abs)

In [ ]:
num_nodes = G_edf.number_of_nodes()
np.random.seed(42) 
coords_edf = np.random.rand(num_nodes, 3) * 10 
edf_degrees = dict(G_edf.degree())
node_size_factor = 50 
node_sizes_edf = [edf_degrees[node] * node_size_factor for node in G_edf.nodes()]
edge_weights_edf = np.array([G_edf[u][v]['weight'] for u, v in G_edf.edges()])
abs_edge_weights_edf = np.abs(edge_weights_edf)
if np.max(abs_edge_weights_edf) != np.min(abs_edge_weights_edf):
    norm_edge_colors_edf = (abs_edge_weights_edf - np.min(abs_edge_weights_edf)) / (np.max(abs_edge_weights_edf) - np.min(abs_edge_weights_edf))
else:
    norm_edge_colors_edf = np.zeros_like(abs_edge_weights_edf) # All weights are the same, use one color
cmap = cm.get_cmap('hot')
edge_colors_edf = cmap(norm_edge_colors_edf)
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
node_xyz_edf = coords_edf
ax.scatter(node_xyz_edf[:, 0], node_xyz_edf[:, 1], node_xyz_edf[:, 2], s=node_sizes_edf, c='skyblue', alpha=0.8)
for i, (u, v) in enumerate(G_edf.edges()):
    x = np.array([coords_edf[u, 0], coords_edf[v, 0]])
    y = np.array([coords_edf[u, 1], coords_edf[v, 1]])
    z = np.array([coords_edf[u, 2], coords_edf[v, 2]])
    ax.plot(x, y, z, color=edge_colors_edf[i], alpha=0.5)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('3DG EDF Connectivity Matrix')
plt.show()